In [17]:
from collections import defaultdict

import pandas as pd
import os

from pandas.core.methods.to_dict import to_dict


def readFiles(FilePath):
    if not os.path.exists(FilePath):
        raise FileNotFoundError(f"Directory not found: {FilePath}")
    dirFiles=os.listdir(FilePath)
    entireHistory=pd.DataFrame()
    for file in dirFiles:
        if file.endswith(".json"):
            entireHistory=pd.concat([entireHistory,pd.read_json(FilePath+file)])
    return entireHistory

def filterHistory(Data):
    processed=Data.drop(["platform","ip_addr","shuffle","offline","offline_timestamp","conn_country","incognito_mode","reason_start","spotify_track_uri","spotify_episode_uri","audiobook_chapter_title","audiobook_chapter_uri","audiobook_title","audiobook_uri","episode_show_name","episode_name","skipped"],axis=1) #removing columns that don't give us any useful information
    processed=processed[~processed["master_metadata_track_name"].isna()] #dropping rows that aren't songs (podcasts, audiobooks etc...)
    processed=processed[processed["ms_played"]>25000]
    processed["ts"]= pd.to_datetime(processed["ts"]).dt.date
    processed.reset_index(inplace=True,drop=True)
    processed.columns=["ts","ms_played","track_name","artist_name","album_name","reason_end"]
    return processed




In [18]:
testPath="../"
try:
    extendedHistory=readFiles(testPath)
except FileNotFoundError:
    print("File not found enter the correct path")


In [19]:
filteredHistory=filterHistory(extendedHistory)

In [15]:
top_artists=filteredHistory.groupby("artist_name")

In [6]:
def generateJson(username,filteredHistory):
    json_structure = {
        "username":username,
        "total_plays":len(filteredHistory),
        "artists":{}
    }
    groupby_artist = filteredHistory.groupby("artist_name")
    for artist,data in groupby_artist:
        artist_data = {
            "total_plays":len(data),
            "tracks":defaultdict(int),
            "albums":defaultdict(int)
        }
        track_plays=data.groupby("track_name").size()
        for track_name, play_count in track_plays.items():
            artist_data["tracks"][track_name]=play_count
        album_plays=data.groupby("album_name").size()
        for album_name, play_count in album_plays.items():
            artist_data["albums"][album_name]=play_count
        json_structure["artists"][artist]=artist_data
    return json_structure

In [17]:
filteredHistory.head()

,ts,ms_played,track_name,artist_name,album_name,reason_end
0,2024-05-31,27561,Bangala Kathamulo,Ramana Gogula,Badri,endplay
1,2024-06-22,29753,Candy Shop,50 Cent,The Massacre,trackdone
2,2024-09-07,29753,カワキヲアメク,美波,カワキヲアメク,trackdone
3,2022-09-12,212578,Never Again,Ryan.B,Never Again,trackdone
4,2022-09-12,178770,放個大招給你看 (OPPO Reno 宣傳曲),Ryan.B,放個大招給你看 (OPPO Reno 宣傳曲),trackdone


In [ ]:
from fastapi import FastAPI, HTTPException
import sqlite3
from typing import List

def saveToSql(filteredData: pd.DataFrame,username: str,db_path: str="spotify_data.db"):
    conn=sqlite3.connect(db_path)
    createTableUnique = """USERS{
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE
    }"""
    conn.execute(createTableUnique)
    cursor = conn.cursor()
    cursor.execute("INSERT OR IGNORE INTO Users (username) VALUES (?)", (username,))
    conn.commit()
    cursor.execute("SELECT id FROM USERS WHERE username = ?", (username,))
    user_id = cursor.fetchone()[0]

    create_table_query="""
    CREATE TABLE IF NOT EXISTS StreamingHistory(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT,
    ts TEXT,
    ms_played INTEGER,
    track_name TEXT,
    artist_name TEXT,
    album_name TEXT,
    reason_end TEXT
    )
    """
    conn.execute(create_table_query)
    filteredData["user_id"]=user_id
    filteredData.drop(columns=["username"], errors="ignore", inplace=True)  # Drop username if it exists

    filteredData.to_sql("StreamingHistory",conn,if_exists="append",index=False)
    conn.close()

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="7a79eb3e972b495ca9ce12142796e2b4",
                                               client_secret="745ab3b3687e4fee91f8ecbf7f4e30ab",
                                               redirect_uri="http://localhost:1110",
                                               scope="user-library-read"))

In [1]:
import sqlite3

con=sqlite3.connect("test.db")
cur=con.cursor()

In [4]:
cur.execute("CREATE TABLE movie(title TEXT,year INTEGER, score INTEGER)")
res=cur.execute("SELECT name from sqlite_master")
res.fetchone()

('movie',)

In [1]:
import sqlite3
conn = sqlite3.connect("spotify_data.db")

In [10]:
cursor = conn.cursor()
query = """
SELECT track_name, COUNT(*) AS play_count
FROM StreamingHistory
WHERE artist_name = 'Joji'
GROUP BY track_name
ORDER BY play_count DESC;
"""
cursor.execute(query)

In [11]:
results=cursor.fetchall()
for row in results:
    print(f"Track: {row[0]}, Play Count: {row[1]}")

Track: Like You Do, Play Count: 102
Track: SLOW DANCING IN THE DARK, Play Count: 90
Track: Afterthought, Play Count: 86
Track: worldstar money (interlude), Play Count: 84
Track: Sanctuary, Play Count: 64
Track: Normal People (feat. rei brown), Play Count: 48
Track: Daylight, Play Count: 28
Track: Glimpse of Us, Play Count: 24
Track: ATTENTION, Play Count: 22
Track: CAN'T GET OVER YOU (feat. Clams Casino), Play Count: 20
Track: Die For You, Play Count: 18
Track: Feeling Like The End, Play Count: 14
Track: Demons, Play Count: 14
Track: 777, Play Count: 14
Track: YEAH RIGHT, Play Count: 12
Track: Will He, Play Count: 12
Track: TEST DRIVE, Play Count: 8
Track: R.I.P. (feat. Trippie Redd), Play Count: 8
Track: Gimme Love, Play Count: 8
Track: Bitter Fuck, Play Count: 8
Track: Your Man, Play Count: 6
Track: YUKON (INTERLUDE), Play Count: 6
Track: Upgrade, Play Count: 6
Track: NITROUS, Play Count: 6
Track: NIGHT RIDER, Play Count: 6
Track: I Don't Wanna Waste My Time, Play Count: 6
Track: Ew,

PermissionError: [Errno 13] Permission denied: '/data'